In [1]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

['Ariel Sharon' 'Colin Powell' 'Donald Rumsfeld' 'George W Bush'
 'Gerhard Schroeder' 'Hugo Chavez' 'Junichiro Koizumi' 'Tony Blair']
(1348, 62, 47)


- **¿Qué hace esta celda?**

La celda descarga y carga el conjunto Labeled Faces in the Wild (LFW), ampliamente utilizado en aprendizaje automático para tareas de reconocimiento facial; a continuación muestra por pantalla los nombres de las personas presentes en el dataset y las dimensiones del conjunto de imágenes.

- **Explicación paso a paso**

fetch_lfw_people obtiene el dataset LFW, que contiene fotografías de rostros para trabajos de reconocimiento facial.

min_faces_per_person=60 aplica un filtro que conserva únicamente a las personas que cuentan con 60 o más imágenes, evitando así clases con pocas muestras.

faces.target_names devuelve la lista de nombres (las clases) incluidas en el conjunto.

faces.images.shape da la forma del arreglo de imágenes; por ejemplo, (1348, 62, 47) significa 1,348 imágenes de tamaño 62×47 píxeles.

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(3, 5)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

- **¿Qué hace esta celda?**

La celda dibuja una cuadrícula de subgráficos para inspeccionar visualmente imágenes del dataset LFW: muestra las primeras 15 caras con sus etiquetas para comprobar la carga de datos, apreciar la variabilidad entre clases y evaluar la calidad de las imágenes antes de entrenar modelos.

- **Explicación paso a paso**

Se importa matplotlib.pyplot y se activa %matplotlib inline para visualizar dentro del notebook; plt.subplots(3, 5) crea una rejilla de 3×5 ejes; en el bucle for se representan las primeras 15 imágenes faces.images[i] en escala de grises (cmap='bone'), se ocultan las marcas de los ejes (xticks=[], yticks=[]) y se coloca el nombre de la persona como título o etiqueta bajo cada imagen.

In [3]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

pca = RandomizedPCA(n_components=150, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

- **¿Qué hace esta celda?**

La celda crea un modelo de reconocimiento facial que combina reducción de dimensionalidad (PCA) con un clasificador SVM: primero transforma las imágenes en un espacio de 150 componentes principales (eigenfaces) para compactar la información y luego entrena una SVM con núcleo radial sobre esas componentes, lo que reduce ruido y mejora la eficacia y velocidad del clasificador.

- **Explicación paso a paso**

PCA(n_components=150) proyecta las imágenes en 150 componentes principales, reteniendo la mayor parte de la varianza y reduciendo la dimensionalidad de miles de píxeles a 150 características.

whiten=True escala las componentes para que tengan varianza unitaria (descorrelaciona y normaliza), lo que suele ayudar a estabilidad y rendimiento del clasificador.

SVC(kernel='rbf') define un clasificador SVM con kernel radial (RBF), capaz de separar clases que no son linealmente separables.

class_weight='balanced' ajusta automáticamente los pesos de cada clase en la función de pérdida según su frecuencia, mitigando el impacto de clases desbalanceadas.

make_pipeline(pca, svc) encadena PCA y SVM en un pipeline único que aplica la reducción de dimensión y, acto seguido, el entrenamiento/predicción con la SVM, facilitando validación cruzada y uso reproducible.

In [4]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target, random_state=42)

- **¿Qué hace esta celda?**

La celda divide el conjunto de imágenes y sus etiquetas en dos subconjuntos: uno para entrenar el modelo y otro para evaluar su rendimiento en datos no vistos, de modo que podamos medir la capacidad de generalización del clasificador.

- **Explicación paso a paso**

Se usa train_test_split para separar faces.data (las imágenes aplanadas) y faces.target (las etiquetas) en conjuntos de entrenamiento y prueba; por defecto la función reserva el 75% de los datos para entrenamiento y el 25% para prueba; random_state=42 fija la semilla del generador aleatorio para que la partición sea reproducible en ejecuciones posteriores.

In [5]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)

CPU times: total: 55.1 s
Wall time: 43.4 s
{'svc__C': 5, 'svc__gamma': 0.001}


- **¿Qué hace esta celda?**

La celda realiza una búsqueda exhaustiva de hiperparámetros para el clasificador SVM dentro del pipeline, con el fin de encontrar los valores de C y gamma que maximicen la precisión mediante validación cruzada.

- **Explicación paso a paso**

Se define param_grid con las combinaciones de hiperparámetros a probar: C, que regula la penalización por errores (regularización), y gamma, que determina la influencia de cada punto en el kernel RBF; los prefijos svc__ indican que estos parámetros corresponden al SVM dentro del pipeline. Luego, GridSearchCV evalúa todas las combinaciones usando validación cruzada para identificar la mejor configuración, que se consulta con grid.best_params_. Finalmente, %time muestra cuánto tarda en completarse todo el proceso.

In [6]:
model = grid.best_estimator_
yfit = model.predict(Xtest)

- **¿Qué hace esta celda?**

La celda utiliza el mejor modelo obtenido de GridSearchCV para realizar predicciones sobre los datos de prueba, permitiendo evaluar el rendimiento final del clasificador con los hiperparámetros óptimos.

- **Explicación paso a paso**

grid.best_estimator_ recupera el modelo con la mejor combinación de C y gamma encontrada en la búsqueda; luego, model.predict(Xtest) aplica este modelo optimizado para predecir las etiquetas de las imágenes de prueba; finalmente, yfit almacena las clases predichas para cada rostro en Xtest.

In [7]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')

- **¿Qué hace esta celda?**

La celda visualiza una muestra de imágenes de prueba junto con sus predicciones, mostrando en negro las etiquetas correctas y en rojo las incorrectas, para evaluar de manera rápida y visual el desempeño del modelo.

- **Explicación paso a paso**

Se crea una cuadrícula de 4×6 subgráficos (plt.subplots(4, 6)) para mostrar 24 imágenes; cada imagen de Xtest se reconvierte a su tamaño original (62×47 píxeles) y se dibuja con su etiqueta predicha (yfit[i]); si la predicción coincide con la etiqueta real (ytest[i]), el texto aparece en negro, y si no coincide, en rojo, permitiendo identificar de un vistazo los aciertos y errores del clasificador.

In [8]:

from sklearn.metrics import classification_report
print(classification_report(ytest, yfit,
                            target_names=faces.target_names))

                   precision    recall  f1-score   support

     Ariel Sharon       0.65      0.87      0.74        15
     Colin Powell       0.83      0.88      0.86        68
  Donald Rumsfeld       0.70      0.84      0.76        31
    George W Bush       0.97      0.80      0.88       126
Gerhard Schroeder       0.76      0.83      0.79        23
      Hugo Chavez       0.93      0.70      0.80        20
Junichiro Koizumi       0.86      1.00      0.92        12
       Tony Blair       0.82      0.98      0.89        42

         accuracy                           0.85       337
        macro avg       0.82      0.86      0.83       337
     weighted avg       0.86      0.85      0.85       337



- **¿Qué hace esta celda?**

La celda genera un informe de clasificación que resume el desempeño del modelo usando métricas clave como precisión, exhaustividad y F1-score, permitiendo evaluar qué tan bien el clasificador identifica cada rostro.

- **Explicación paso a paso**

Se utiliza classification_report para comparar las etiquetas reales (ytest) con las predichas (yfit) y calcular métricas por clase: precisión (proporción de predicciones correctas sobre las positivas), exhaustividad/recall (capacidad del modelo para encontrar todas las instancias reales) y F1-score (media armónica entre precisión y recall). El parámetro target_names asocia estas métricas con los nombres reales de cada persona en el dataset.

In [9]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=faces.target_names,
            yticklabels=faces.target_names)

<Axes: >

- **¿Qué hace esta celda?**

La celda visualiza una matriz de confusión, mostrando de forma gráfica cómo clasifica el modelo cada rostro y permitiendo identificar fácilmente las clases donde comete errores.

- **Explicación paso a paso**

confusion_matrix(ytest, yfit) crea una tabla donde las filas representan las clases reales y las columnas las predicciones; luego, sns.heatmap dibuja esta matriz como un mapa de calor, con los números de aciertos y errores (annot=True), y las etiquetas de los ejes (xticklabels y yticklabels) corresponden a los nombres de las personas. Los valores altos en la diagonal indican que el modelo clasificó correctamente, mientras que los valores fuera de la diagonal muestran confusiones entre clases.

## **Dataset load_digits**

### 
Cargamos el conjunto **Digits** y mostramos los nombres de clase y la forma de las imágenes.


In [10]:
from sklearn.datasets import load_digits
digits = load_digits()
print(getattr(digits, "target_names", list(range(10))))
print(digits.images.shape)


[0 1 2 3 4 5 6 7 8 9]
(1797, 8, 8)


### Visualización de ejemplos del dataset
Mostramos una cuadrícula de imágenes para inspección visual rápida (mismas convenciones que antes).


In [12]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(3, 5)
for i, axi in enumerate(ax.flat):
    axi.imshow(digits.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=str(digits.target[i]))


### Creación del modelo con PCA y SVM
Construimos el **pipeline** con PCA (blanqueado) + SVC (RBF), igual que antes.


In [13]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline

pca = RandomizedPCA(n_components=40, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)


### División del conjunto de datos
Separación en **entrenamiento** y **prueba** con semilla fija (por defecto 75/25).


In [14]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(digits.data, digits.target, random_state=42)


### Búsqueda de hiperparámetros con GridSearchCV
Optimizamos `C` y `gamma` del SVC dentro del mismo pipeline (mismo grid y prefijos).


In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid)

%time grid.fit(Xtrain, ytrain)
print(grid.best_params_)


CPU times: total: 28.3 s
Wall time: 27.6 s
{'svc__C': 5, 'svc__gamma': 0.005}


### Predicción con el mejor modelo
Usamos el mejor estimador hallado por la búsqueda para predecir en el set de prueba.


In [16]:
model = grid.best_estimator_
yfit = model.predict(Xtest)


### Visualización de predicciones del modelo
Mostramos una cuadrícula de imágenes de prueba con la **predicción**; negro si acierta y rojo si falla.


In [17]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(8, 8), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(str(yfit[i]),
                   color='black' if yfit[i] == ytest[i] else 'red')


### Evaluación con classification_report
Informe con **precision**, **recall** y **f1-score** por clase y promedios.


In [18]:
from sklearn.metrics import classification_report
print(classification_report(ytest, yfit))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        37
           2       1.00      1.00      1.00        38
           3       1.00      0.96      0.98        46
           4       1.00      1.00      1.00        55
           5       0.97      1.00      0.98        59
           6       1.00      0.98      0.99        45
           7       1.00      0.98      0.99        41
           8       0.95      1.00      0.97        38
           9       0.98      0.98      0.98        48

    accuracy                           0.99       450
   macro avg       0.99      0.99      0.99       450
weighted avg       0.99      0.99      0.99       450



### Matriz de confusión
Mapa de calor para ver aciertos (diagonal) y confusiones entre dígitos.


In [19]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, yfit)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=getattr(digits, "target_names", list(range(10))),
            yticklabels=getattr(digits, "target_names", list(range(10))))


<Axes: >